<a href="https://colab.research.google.com/github/KyleH2997/FootballPredictiorDNNClass/blob/master/dnnattempt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow-gpu==1.15.0

     |████████████████████████████████| 411.5MB 37kB/s 
     |████████████████████████████████| 3.8MB 37.2MB/s 
     |████████████████████████████████| 512kB 39.6MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=3bc5f5662b962129b26c84f2a3d20a98ce0d21fea19cfc9a00df3b13de27c436
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.3.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement tensorflow-estimator<2.4.0,>=2.3.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing i

In [ ]:
import tensorflow as tf
import pandas as pd
from google.colab import files

import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
df = pd.read_csv("Difference.csv")
df.head()

,FTR,HTV,ATV,DIFH
0,1,87.368182,68.563636,18.804545
1,2,62.936364,78.050909,-15.114545
2,0,66.818182,72.527273,-5.709091
3,1,71.390909,59.363636,12.027273
4,0,70.266364,66.845455,3.420909


In [ ]:
train_df = df.sample(frac=0.75, random_state=0)
test_df = df.drop(train_df.index)

In [ ]:
trainy_df = train_df.pop('FTR')
testy_df = test_df.pop('FTR')

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    features = {'HTV': np.array([70.0, 75.0, 58.0]),
                'ATV':  np.array([70.4, 58.0, 75.0]),
                'DIFH': np.array([-0.4, 7.0, -7.0]),}
    labels = np.array([0,1,2])
    return features

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [ ]:
def create_feature_column():
    
    feat_HTV = tf.feature_column.numeric_column('HTV')
    feat_ATV = tf.feature_column.numeric_column('ATV')
    feat_DIFH = tf.feature_column.numeric_column('DIFH')

    feature_column = [feat_HTV, feat_ATV, feat_DIFH] 
    
    return feature_column

In [ ]:
my_feature_columns = create_feature_column()

In [ ]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
   feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[10],
    # The model must choose between 3 classes.
    n_classes=3,
        optimizer=tf.train.ProximalAdagradOptimizer(
            learning_rate=0.1,
            l1_regularization_strength=0.001
        ))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmps54u8asl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f57f07f7518>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train_df, trainy_df, training=True),
    steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmps54u8asl/model.ckpt.
INFO:tensorflow:loss = 7005.574, step = 1
INFO:tensorflow:global_step/sec: 415.1
INFO:tensorflow:loss = 230.98672, step = 101 (0.246 sec)
INFO:tensorflow:global_step/sec: 487.6
INFO:tensorflow:loss = 230.90656, step = 201 (0.205 sec)
INFO:tensorflow:global_step/sec: 417.252
INFO:tensorflow:loss = 233.90588, ste

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test_df, testy_df, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-08-12T12:52:12Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmps54u8asl/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-08-12-12:52:12
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.58947366, average_loss = 0.96571875, global_step = 5000, loss = 91.74328
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmps54u8asl/model.ckpt-5000

Test set accuracy: 0.589



In [ ]:
# Generate predictions from the model
expected = ['Win']
predict_x = {
    'HTV': [50.3681818181818],
    'ATV': [51.8636363636363],
    'DIFH': [-1.58045454545454],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

In [ ]:
RESULTS = ['Draw','HomeWin','AwayWin']

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
    RESULTS[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmps54u8asl/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "AwayWin" (34.7%), expected "Win"


In [ ]:
feature_spec = tf.feature_column.make_parse_example_spec(my_feature_columns);
export_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec);

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.


In [ ]:
classifier.export_savedmodel("saved_model", export_input_fn, as_text=False)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: ['serving_default', 'classification']
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from /tmp/tmps54u8asl/model.ckpt-5000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: saved_model/temp-b'1597238368'/saved_model.pb


b'saved_model/1597238368'

In [90]:
predict_model = tf.contrib.predictor.from_saved_model("/content/saved_model/1597238368")

INFO:tensorflow:Restoring parameters from /content/saved_model/1597238368/variables/variables


In [98]:
predicttest = tf.contrib.predictor.from_saved_model("/content/saved_model/1597238368")

INFO:tensorflow:Restoring parameters from /content/saved_model/1597238368/variables/variables


In [95]:
# Generate predictions from the model
expected = ['Win']
predict_x = {
    'HTV': [52.3681818181818],
    'ATV': [65.8636363636363],
    'DIFH': [-13.58045454545454],
}

def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

In [96]:
predict_model = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

In [97]:
RESULTS = ['Draw','HomeWin','AwayWin']

for pred_dict, expec in zip(predict_model, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
    RESULTS[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmps54u8asl/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "AwayWin" (73.9%), expected "Win"


In [104]:
inputs = pd.DataFrame({
    'HTV': [70.3681818181818],
    'ATV': [55.8636363636363],
    'DIFH': [12.58045454545454],
})

examples = []
for index, row in inputs.iterrows():
    feature = {}
    for col, value in row.iteritems():
        feature[col] = tf.train.Feature(float_list=tf.train.FloatList(value=[value]))
    example = tf.train.Example(
        features=tf.train.Features(
            feature=feature
        )
    )
    examples.append(example.SerializeToString())

In [105]:
predictions = predicttest({'inputs': examples})

In [106]:
predictions

{'classes': array([[b'0', b'1', b'2']], dtype=object),
 'scores': array([[0.09630592, 0.85719633, 0.04649781]], dtype=float32)}